In [8]:
include("benchFunctions.jl")


cols_within_max_Throughput (generic function with 2 methods)

In [12]:
example_filepath = "resuts_adjoint_GH200_f32.csv";
my_df = read_special_csv(example_filepath);
# println("DataFrame letto con successo:")
# println(my_df)

# required_cols = ["cluster_size", "tet_per_block", "nelements", "clock", "Throughput"]
# missing_cols = setdiff(required_cols, names(my_df))

# println("Colonne richieste mancanti: ", missing_cols)
# println("Colonne presenti nel DataFrame: ", names(my_df))

# grouped = groupby(my_df, [:cluster_size, :tet_per_block])


In [17]:

summary_df = summarize_cluster_stats(my_df);
max_row = col_with_max_Throughput(summary_df);
max_row_top = cols_within_max_Throughput(summary_df, 15);


println("Top 15 righe con Throughput massimo:")
println(max_row_top);

Top 15 righe con Throughput massimo:
15×10 DataFrame
 Row │ cluster_size  tet_per_block  min_clock  max_clock  mean_clock  se_clock     min_Throughput  max_Throughput  mean_Throughput  se_Throughput  
     │ Int64         Int64          Float64    Float64    Float64     Float64      Float64         Float64         Float64          Float64        
─────┼─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
   1 │            1              4   0.153991   0.156251    0.154588  0.000195453       1.59632e8       1.61974e8        1.61351e8       2.0238e5
   2 │            8              4   0.154117   0.156648    0.154978  0.000239128       1.59227e8       1.61842e8        1.60946e8  247199.0
   3 │            4              4   0.150586   0.159856    0.155795  0.000917152       1.56032e8       1.65637e8        1.60149e8       9.47147e5
   4 │           12              4   0.156965   0.159206    0.157705 

In [11]:
summary_df = summarize_cluster_stats(my_df)
println("Cluster stats summary:")
println(summary_df)

Cluster stats summary:
65×10 DataFrame
 Row │ cluster_size  tet_per_block  min_clock  max_clock  mean_clock  se_clock     min_Throughput  max_Throughput  mean_Throughput  se_Throughput  
     │ Int64         Int64          Float64    Float64    Float64     Float64      Float64         Float64         Float64          Float64        
─────┼─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
   1 │            1              4   0.153991   0.156251    0.154588  0.000195453       1.59632e8       1.61974e8        1.61351e8       2.0238e5
   2 │            1              6   0.163218   0.165091    0.163592  0.000170815       1.51084e8       1.52817e8        1.5247e8        1.57999e5
   3 │            1              8   0.176484   0.178641    0.176834  0.000203534       1.39624e8       1.4133e8         1.41052e8       1.60942e5
   4 │            1             12   0.212423   0.214454    0.212886  0.00017